# Tutorial 4: ML on the best [2~50] features

---

### Introduction

Hello, this notebook will show how to use machine learning (ML) to classify apples using the best [2, .. , 50] wavelength. The notebook will show how to visualize the final accuracy and their relative performance in two different graphs.

first, let us call the data

In [1]:
%store -r df_input_GS
%store -r y_GS
%store -r df_GS

In [2]:
y_GS = y_GS.map({'S': 1, 'B': 0})

---

calling some packages

In [3]:
# Python packages 
import pandas as pd # for importing data into data frame format
import seaborn as sns # For drawing useful graphs, such as bar graphs
import numpy as np
import matplotlib.pyplot as plt

---

<b><i> Data splitting </i></b> 

In [4]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split( df_input_GS, y_GS, test_size = 0.3, random_state=3, stratify=y_GS) # train and valid sets

In [5]:
print(Xtrain.shape)
print(Xtest.shape)

(352, 2074)
(151, 2074)


---

<b><i> get the best N features </i></b> 

In [6]:
%store -r featImp_GS

In [7]:
arrimp = np.array(featImp_GS).mean(0)
sorted_idx = arrimp.argsort()
print(sorted_idx[-10:])

[1774   45  118  572   69  842 1781  877   18   52]


In [8]:
cols_ordered = df_input_GS.columns[sorted_idx][-400:]
cols_ordered

Index([ 938.682, 1131.663,  835.258, 2079.102, 1077.573,  1368.87, 2072.455,
       1666.222, 2478.624,    888.5,
       ...
       1940.599,  845.886,  866.524, 1021.529,  852.562,  1143.14, 1950.821,
       1161.057,    838.5,  847.823],
      dtype='object', length=400)

---

<b><i> prepare diffrint feature set sizes </i></b> 

In [9]:
number_features = [2, 4, 6, 8, 10, 20, 50] # define the number features. 


---

<b><i> ML model and thier hyper-parameters </i></b> 

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from numpy import random as np_random

first let us define the model


In [11]:
lr = LogisticRegression()
svm = SVC(kernel="rbf")
rf = RandomForestClassifier(random_state=1)
xgb = XGBClassifier(random_state=1)
knns = KNeighborsClassifier()
anns = MLPClassifier(max_iter=3000, random_state=1, activation = 'relu', solver = 'sgd')

defining the hyper-paramter per model

In [12]:
solvers = ['newton-cg', 'liblinear']
penalty = ['l2']
c_values = [1000, 100, 10, 1.0, 0.1, 0.01, 0.001]
lr_par = dict(solver=solvers,penalty=penalty,C=c_values)

##############################
C_range = np.logspace(-1, 10, 20) # define a set of values for the parameter C
gamma_range = np.logspace(-9, 3, 20) # define a set of values for the parameter gamma

svm_par = dict(gamma=gamma_range, C=C_range)

#########################

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid

#max_features = ['sqrt', 'log2']

rf_par = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
#######################
# https://www.kaggle.com/code/tilii7/hyperparameter-grid-search-with-xgboost/notebook
xgb_par = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5, 9],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 8]
        }
##################################
n_neighbors = range(1, 30, 2)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']

knns_par = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)
################
anns_par = {
    'hidden_layer_sizes': [(3,3),(5,5),(8,3)],
#     'activation': ['relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.001,0.01,0.1, 0.05, 0.005 , 0.005, 0.00005],
#     'learning_rate': ['constant','adaptive'],
}
################

group the above into two lists

In [13]:
models = [[lr, 'lr', 14], [svm, 'svm', 15], [rf, 'rf', 30], [xgb, 'xgb', 30], [knns, 'knns', 30], [anns, 'anns', 30]]
par = [lr_par, svm_par, rf_par, xgb_par, knns_par, anns_par]

---

<b><i> train and test </i></b> 

In [14]:
from source.ml_acc import get_accuracy_ml

In [15]:
num_ml_tools = len(par)
ml_dicts = {}

for m, par in zip(models, par):
    key0 = str(m[1])
    ml_dicts[key0] = {}
    for f in number_features:
        print(f)
        print(cols_ordered[-f:])
        print(type(Xtrain))
        xtr =  Xtrain[cols_ordered[-f:]]
        xte =  Xtest[cols_ordered[-f:]]
        results = get_accuracy_ml (m[0], m[2], par, np.array(xtr), np.array(Ytrain), np.array(xte), np.array(Ytest)) # to get the accuracies for the ml model

        key = str(m[1])+","+str((f))
        ml_dicts[key0][key] = {}

        ml_dicts[key0][key]['tot_acc'] = results[0]
        ml_dicts[key0][key]['jack_train'] = results[1]
        ml_dicts[key0][key]['jack_test'] = results[2]

2
Index([838.5, 847.823], dtype='object')
<class 'pandas.core.frame.DataFrame'>


/users/hussein/source/apple_classification/appleVer/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/hussein/source/apple_classification/appleVer/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/hussein/source/apple_classification/appleVer/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/

4
Index([1950.821, 1161.057, 838.5, 847.823], dtype='object')
<class 'pandas.core.frame.DataFrame'>
6
Index([852.562, 1143.14, 1950.821, 1161.057, 838.5, 847.823], dtype='object')
<class 'pandas.core.frame.DataFrame'>
8
Index([866.524, 1021.529, 852.562, 1143.14, 1950.821, 1161.057, 838.5,
       847.823],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
10
Index([1940.599,  845.886,  866.524, 1021.529,  852.562,  1143.14, 1950.821,
       1161.057,    838.5,  847.823],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
20
Index([1021.932,  836.606,  844.784,  855.375,  862.202,  849.489,  835.796,
         968.85,  833.915, 1968.596, 1940.599,  845.886,  866.524, 1021.529,
        852.562,  1143.14, 1950.821, 1161.057,    838.5,  847.823],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
50
Index([ 852.282, 1891.058,  871.183, 1128.707,  853.123,  836.875,   842.04,
        854.247, 1147.694, 1354.567,  965.243, 1906.353,  863.638,  838.229,
        952.

KeyboardInterrupt: 

In [ ]:
import json

In [ ]:
with open('ml_gs.txt', 'w') as file:
     file.write(json.dumps(ml_dicts)) # use `json.loads` to do the reverse

In [ ]:
print(1)